In [41]:
%load_ext autoreload
%autoreload 2

import numpy as np
from datasets import load_dataset


dataset = load_dataset(
    "oscar-corpus/OSCAR-2201",
    token=True,  # required
    language="id",
    streaming=True,  # optional
    split="train",
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from lib import process, get_ngram_freq_dict
from tqdm import tqdm

characters_cnt = np.zeros((128), dtype=np.int64)
bigrams_cnt = np.zeros((128, 128), dtype=np.int64)
skipgrams_cnt = np.zeros((128, 128), dtype=np.int64)
skipgrams2_cnt = np.zeros((128, 128), dtype=np.int64)
skipgrams3_cnt = np.zeros((128, 128), dtype=np.int64)
trigrams_cnt = np.zeros((128, 128, 128), dtype=np.int64)

for d in tqdm(dataset, total=2244622):
    id = d["id"]
    text = d["text"]
    process(
        text,
        characters_cnt,
        bigrams_cnt,
        skipgrams_cnt,
        skipgrams2_cnt,
        skipgrams3_cnt,
        trigrams_cnt,
    )
    

  0%|          | 0/708000 [00:00<?, ?it/s]

2244622it [11:23:21, 54.74it/s]                             


In [43]:
config = [
    (characters_cnt, "characters"),
    (bigrams_cnt, "bigrams"),
    (skipgrams_cnt, "skipgrams"),
    (skipgrams2_cnt, "skipgrams2"),
    (skipgrams3_cnt, "skipgrams3"),
    (trigrams_cnt, "trigrams"),
]

stats = dict({})
for ngram_cnt, stats_key  in config:
    ngram_freq = get_ngram_freq_dict(ngram_cnt)
    stats[stats_key] = ngram_freq


In [44]:
import json
import os

stats_path = "static/stats/oscar/id.raw.json"
os.makedirs(os.path.dirname(stats_path), exist_ok=True)
with open(stats_path, "w") as f:
    json.dump(stats, f, indent=2)